In [ ]:
TOKEN="<redact>"

In [ ]:
!oc login \
    --token="{TOKEN}" \
    --server=<redact>

In [ ]:
!oc create secret generic raft-workshop-secrets \
  --from-literal=S3_ENDPOINT="http://minio-api.minio-operator.svc.cluster.local:9000" \
  --from-literal=S3_BUCKET="test" \
  --from-literal=S3_ACCESS_KEY="<redact>" \
  --from-literal=S3_SECRET_KEY="<redact>"

In [ ]:
%%bash
oc create -f - <<EOF
apiVersion: batch/v1
kind: Job
metadata:
  generateName: granite-ft-surfing-
  labels:
    app: granite-ft-surfing
spec:
  backoffLimit: 0
  template:
    metadata:
      labels:
        app: granite-ft
    spec:
      restartPolicy: Never
      serviceAccountName: default
      containers:
        - name: granite-ft-trainer
          image: docker.io/cengleby86/peft-training-workshop:latest
          imagePullPolicy: Always
          command: ["python"]
          args: ["train.py"]
          envFrom:
            - secretRef:
                name: raft-workshop-secrets
          env:
            - name: S3_NAMESPACE
              valueFrom:
                fieldRef:
                  fieldPath: metadata.namespace
            - name: DATASET_NAME
              value: "surfing"
          resources:
            limits:
              nvidia.com/gpu: "1"
              cpu: "4"
              memory: "20Gi"
            requests:
              cpu: "1"
              memory: "8Gi"
          volumeMounts:
            - name: tmp-volume
              mountPath: /tmp
      volumes:
        - name: tmp-volume
          emptyDir:
            sizeLimit: "40Gi"
      nodeSelector:
        nvidia.com/gpu.present: "true"
EOF